# Distill `MongoDB/mdbr-leaf-mt` → OpenAI `text-embedding-3-large` (1536-d)

**Goal**: Reproduce [Kabumbus/mdbr-leaf-mt-1536-onnx](https://huggingface.co/Kabumbus/mdbr-leaf-mt-1536-onnx) but targeting `text-embedding-3-large` embeddings instead of `text-embedding-3-small`, using the [Qdrant/dbpedia-entities-openai3-text-embedding-3-large-1536-1M](https://huggingface.co/datasets/Qdrant/dbpedia-entities-openai3-text-embedding-3-large-1536-1M) dataset.

**Architecture**:
- **Student**: `MongoDB/mdbr-leaf-mt` (23M params, BERT-based SentenceTransformer)
- **Teacher labels**: Pre-computed OpenAI `text-embedding-3-large` embeddings (1536-d)
- **Loss**: Cosine regression — `loss = 1 - mean(cosine_similarity(student, teacher))`
- **Partial fine-tuning**: Only the Dense projection head + last 2 transformer blocks are trainable

**Resilience**: All checkpoints, processed data, and training state are saved to Google Drive, so training survives Colab disconnections.

## 0. Setup & Install Dependencies

In [ ]:
!pip install -q sentence-transformers datasets transformers accelerate onnx onnxruntime optimum[onnxruntime] matplotlib

In [ ]:
# ── Mount Google Drive for persistent storage ──────────────────────────────
from google.colab import drive
drive.mount("/content/drive")

import os, json, shutil
from pathlib import Path

# All persistent artefacts live here
DRIVE_ROOT = Path("/content/drive/MyDrive/distil_mdbr_large_1536")
DRIVE_ROOT.mkdir(parents=True, exist_ok=True)

CHECKPOINT_DIR = DRIVE_ROOT / "checkpoints"
PROCESSED_DATA = DRIVE_ROOT / "processed_dataset"
FINAL_MODEL_DIR = DRIVE_ROOT / "final_model"
ONNX_DIR = DRIVE_ROOT / "onnx_export"
STATE_FILE = DRIVE_ROOT / "training_state.json"

for d in [CHECKPOINT_DIR, PROCESSED_DATA, FINAL_MODEL_DIR, ONNX_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"✅ Drive mounted. Working directory: {DRIVE_ROOT}")

## 1. Load & Prepare Dataset

The dataset has columns: `_id`, `title`, `text`, `openai` (the 1536-d teacher embedding).
We combine `title + " " + text` into a single `text` field and rename `openai` → `labels`.

In [ ]:
from datasets import load_dataset, load_from_disk, Features, Value, Sequence
import numpy as np

# ── Check if we already prepared & saved the dataset ───────────────────────
if (PROCESSED_DATA / "dataset_dict.json").exists():
    print("📂 Loading pre-processed dataset from Drive…")
    from datasets import DatasetDict
    ds = DatasetDict.load_from_disk(str(PROCESSED_DATA))
    print(f"   Train: {len(ds['train']):,}  |  Eval: {len(ds['test']):,}")
else:
    print("⬇️  Downloading Qdrant/dbpedia-entities-openai3-text-embedding-3-large-1536-1M …")
    raw = load_dataset(
        "Qdrant/dbpedia-entities-openai3-text-embedding-3-large-1536-1M",
        split="train",
    )
    print(f"   Raw rows: {len(raw):,}")
    print(f"   Columns : {raw.column_names}")
    print(f"   Example keys: {list(raw[0].keys())}")

    # ── Combine title + text → single 'text' field; rename openai → labels ─
    def preprocess(example):
        title = (example.get("title") or "").strip()
        body  = (example.get("text")  or "").strip()
        combined = f"{title} {body}".strip() if title else body
        return {"text": combined, "labels": example["openai"]}

    raw = raw.map(preprocess, remove_columns=raw.column_names, num_proc=4,
                  desc="Preprocessing")

    # Cast labels to fixed-length float32 sequence
    new_features = Features({
        "text":   Value("string"),
        "labels": Sequence(Value("float32"), length=1536),
    })
    raw = raw.cast(new_features)

    # ── Shuffle & split: 99 % train / 1 % eval ────────────────────────────
    raw = raw.shuffle(seed=42)
    split = raw.train_test_split(test_size=0.01, seed=42)

    print(f"   Train: {len(split['train']):,}  |  Eval: {len(split['test']):,}")

    # ── Persist to Drive ───────────────────────────────────────────────────
    split.save_to_disk(str(PROCESSED_DATA))
    ds = split
    print("💾 Dataset saved to Drive.")

print(f"\n✅ Dataset ready.  Sample text[:120]: {ds['train'][0]['text'][:120]}…")

## 2. Load Base Model & Replace Projection Head

Load `MongoDB/mdbr-leaf-mt` and replace (or append) the Dense projection head to output **1536** dimensions with Xavier-uniform init. Then freeze everything except:
- The Dense head(s)
- The last 2 transformer blocks
- Any final normalisation layer

In [ ]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Dense

TARGET_DIM = 1536
BASE_MODEL_NAME = "MongoDB/mdbr-leaf-mt"

# ── Load base model ───────────────────────────────────────────────────────
model = SentenceTransformer(BASE_MODEL_NAME)
print("Base model modules:")
for i, mod in enumerate(model):
    print(f"  [{i}] {mod.__class__.__name__}  →  {type(mod)}")

# ── Determine the backbone hidden size ─────────────────────────────────────
# The Transformer module's output dim is the backbone hidden size
backbone = model[0]  # Transformer module
hidden_size = backbone.get_word_embedding_dimension()
print(f"\nBackbone hidden size: {hidden_size}")

# ── Replace / append Dense head to project to 1536 ────────────────────────
# Check if a Dense layer already exists
dense_idx = None
for i, mod in enumerate(model):
    if isinstance(mod, Dense):
        dense_idx = i
        break

new_dense = Dense(
    in_features=hidden_size,
    out_features=TARGET_DIM,
    bias=False,
    activation_function=nn.Identity(),
)
# Xavier uniform initialisation
nn.init.xavier_uniform_(new_dense.linear.weight)

if dense_idx is not None:
    print(f"♻️  Replacing existing Dense at index {dense_idx} ({hidden_size} → {TARGET_DIM})")
    model[dense_idx] = new_dense
else:
    print(f"➕ Appending new Dense head ({hidden_size} → {TARGET_DIM})")
    model.append(new_dense)

# ── Freeze all, then selectively unfreeze ──────────────────────────────────
for param in model.parameters():
    param.requires_grad = False

# 1) Unfreeze Dense head(s)
for mod in model:
    if isinstance(mod, Dense):
        for param in mod.parameters():
            param.requires_grad = True

# 2) Unfreeze last 2 transformer encoder blocks
encoder = backbone.auto_model
# Works for BERT-style models (encoder.layer is a ModuleList)
if hasattr(encoder, "encoder") and hasattr(encoder.encoder, "layer"):
    layers = encoder.encoder.layer
elif hasattr(encoder, "layer"):
    layers = encoder.layer
else:
    raise RuntimeError(f"Cannot find transformer layers in {type(encoder)}")

num_layers = len(layers)
for layer in layers[-2:]:
    for param in layer.parameters():
        param.requires_grad = True
print(f"🔓 Unfroze last 2 of {num_layers} transformer blocks")

# 3) Unfreeze any final LayerNorm / Normalize module
for mod in model:
    mod_name = mod.__class__.__name__.lower()
    if "norm" in mod_name:
        for param in mod.parameters():
            param.requires_grad = True
        print(f"🔓 Unfroze normalisation module: {mod.__class__.__name__}")

# Enable gradient checkpointing on the backbone
if hasattr(encoder, "gradient_checkpointing_enable"):
    encoder.gradient_checkpointing_enable()
    print("✅ Gradient checkpointing enabled")

# ── Summary ────────────────────────────────────────────────────────────────
total  = sum(p.numel() for p in model.parameters())
train  = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen = total - train
print(f"\nTotal params:     {total:>12,}")
print(f"Trainable params: {train:>12,}  ({100*train/total:.1f}%)")
print(f"Frozen params:    {frozen:>12,}  ({100*frozen/total:.1f}%)")
print(f"Output dim:       {model.get_sentence_embedding_dimension()}")

## 3. Training with Cosine Regression Loss

**Loss**: `1 − mean(cosine_similarity(student_emb, teacher_emb))`

**Key hyperparameters** (matching the reference):
- Epochs: 10
- LR: 1e-3 (cosine schedule, 3% warmup)
- Weight decay: 0.01
- Optimizer: AdamW (fused)
- BF16/FP16 auto-selection
- Checkpoints saved to Drive every 500 steps

In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import get_cosine_schedule_with_warmup
import time, math, glob

# ═══════════════════════════════════════════════════════════════════════════
# Custom Dataset wrapper
# ═══════════════════════════════════════════════════════════════════════════
class EmbeddingDistillDataset(Dataset):
    """Wraps an HF dataset with columns 'text' and 'labels'."""
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset[idx]
        return row["text"], torch.tensor(row["labels"], dtype=torch.float32)


# ═══════════════════════════════════════════════════════════════════════════
# Collate function — tokenise a batch of texts
# ═══════════════════════════════════════════════════════════════════════════
tokenizer = model.tokenizer

def collate_fn(batch):
    texts, labels = zip(*batch)
    encoded = tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    return encoded, torch.stack(labels)


# ═══════════════════════════════════════════════════════════════════════════
# Cosine regression loss
# ═══════════════════════════════════════════════════════════════════════════
def cosine_loss(student_emb, teacher_emb):
    """1 − mean(cosine_similarity(student, teacher))"""
    s = F.normalize(student_emb, p=2, dim=-1)
    t = F.normalize(teacher_emb, p=2, dim=-1)
    return 1.0 - (s * t).sum(dim=-1).mean()


# ═══════════════════════════════════════════════════════════════════════════
# Hyperparameters
# ═══════════════════════════════════════════════════════════════════════════
EPOCHS        = 10
BATCH_SIZE    = 64        # Adjust for GPU memory (T4=64, A100=256)
LR            = 1e-3
WEIGHT_DECAY  = 0.01
WARMUP_RATIO  = 0.03
SAVE_STEPS    = 500       # Checkpoint every N steps
LOG_STEPS     = 50        # Log every N steps
MAX_GRAD_NORM = 1.0

# Auto-select precision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_bf16 = device.type == "cuda" and torch.cuda.is_bf16_supported()
use_fp16 = device.type == "cuda" and not use_bf16
scaler = torch.amp.GradScaler("cuda", enabled=use_fp16)
amp_dtype = torch.bfloat16 if use_bf16 else torch.float16
print(f"Device: {device}  |  AMP dtype: {amp_dtype}  |  Batch size: {BATCH_SIZE}")

# ═══════════════════════════════════════════════════════════════════════════
# DataLoaders
# ═══════════════════════════════════════════════════════════════════════════
train_dataset = EmbeddingDistillDataset(ds["train"])
eval_dataset  = EmbeddingDistillDataset(ds["test"])

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    collate_fn=collate_fn, num_workers=2, pin_memory=True, drop_last=True,
)
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE * 2, shuffle=False,
    collate_fn=collate_fn, num_workers=2, pin_memory=True,
)

steps_per_epoch = len(train_loader)
total_steps = steps_per_epoch * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)
print(f"Steps/epoch: {steps_per_epoch:,}  |  Total steps: {total_steps:,}  |  Warmup: {warmup_steps:,}")

# ═══════════════════════════════════════════════════════════════════════════
# Optimizer & Scheduler
# ═══════════════════════════════════════════════════════════════════════════
trainable_params = [p for p in model.parameters() if p.requires_grad]
try:
    optimizer = torch.optim.AdamW(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY, fused=True)
    print("Using fused AdamW")
except TypeError:
    optimizer = torch.optim.AdamW(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY)
    print("Using standard AdamW (fused not available)")

scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)

# ═══════════════════════════════════════════════════════════════════════════
# Resume from checkpoint if available
# ═══════════════════════════════════════════════════════════════════════════
start_epoch = 0
global_step = 0
train_losses = []
eval_losses  = []

def find_latest_checkpoint():
    """Find the latest checkpoint directory on Drive."""
    pattern = str(CHECKPOINT_DIR / "step_*")
    ckpts = sorted(glob.glob(pattern), key=lambda x: int(x.split("step_")[-1]))
    return Path(ckpts[-1]) if ckpts else None

latest_ckpt = find_latest_checkpoint()
if latest_ckpt is not None:
    print(f"\n🔄 Resuming from checkpoint: {latest_ckpt.name}")
    ckpt = torch.load(latest_ckpt / "training_state.pt", map_location=device, weights_safe_only=False)
    model.load_state_dict(ckpt["model_state_dict"])
    optimizer.load_state_dict(ckpt["optimizer_state_dict"])
    scheduler.load_state_dict(ckpt["scheduler_state_dict"])
    if use_fp16:
        scaler.load_state_dict(ckpt["scaler_state_dict"])
    start_epoch  = ckpt["epoch"]
    global_step  = ckpt["global_step"]
    train_losses = ckpt.get("train_losses", [])
    eval_losses  = ckpt.get("eval_losses", [])
    print(f"   Resumed at epoch {start_epoch}, global step {global_step}")
else:
    print("\n🆕 Starting training from scratch.")

model.to(device)
print("✅ Model moved to device.")

### 3b. Training Loop (interrupt-safe)

Every `SAVE_STEPS` steps the full training state (model, optimizer, scheduler, scaler, loss history) is saved to Google Drive. If the Colab runtime disconnects, just re-run all cells — training resumes from the last checkpoint automatically.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# Evaluation helper
# ═══════════════════════════════════════════════════════════════════════════
@torch.no_grad()
def evaluate():
    model.eval()
    total_loss, n_batches = 0.0, 0
    for encoded, labels in eval_loader:
        encoded = {k: v.to(device) for k, v in encoded.items()}
        labels = labels.to(device)
        with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=(device.type == "cuda")):
            output = model(encoded)
            emb = output["sentence_embedding"]
            loss = cosine_loss(emb, labels)
        total_loss += loss.item()
        n_batches += 1
    model.train()
    return total_loss / max(n_batches, 1)


# ═══════════════════════════════════════════════════════════════════════════
# Checkpoint saver
# ═══════════════════════════════════════════════════════════════════════════
def save_checkpoint(epoch, step, extra_tag=""):
    tag = f"step_{step}" + (f"_{extra_tag}" if extra_tag else "")
    ckpt_path = CHECKPOINT_DIR / tag
    ckpt_path.mkdir(parents=True, exist_ok=True)

    state = {
        "epoch": epoch,
        "global_step": step,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "scaler_state_dict": scaler.state_dict() if use_fp16 else {},
        "train_losses": train_losses,
        "eval_losses": eval_losses,
    }
    torch.save(state, ckpt_path / "training_state.pt")

    # Keep only last 3 checkpoints to save space
    all_ckpts = sorted(glob.glob(str(CHECKPOINT_DIR / "step_*")),
                       key=lambda x: int(x.split("step_")[-1].split("_")[0]))
    for old in all_ckpts[:-3]:
        shutil.rmtree(old, ignore_errors=True)

    print(f"   💾 Checkpoint saved: {tag}")


# ═══════════════════════════════════════════════════════════════════════════
# Training loop
# ═══════════════════════════════════════════════════════════════════════════
print(f"\n{'='*70}")
print(f"  TRAINING START  —  Epochs {start_epoch}→{EPOCHS}  |  Global step: {global_step}")
print(f"{'='*70}\n")

model.train()
t0 = time.time()
running_loss = 0.0

for epoch in range(start_epoch, EPOCHS):
    epoch_step = 0
    for batch_idx, (encoded, labels) in enumerate(train_loader):
        # Skip already-completed steps within the current epoch when resuming
        current_abs_step = epoch * steps_per_epoch + batch_idx
        if current_abs_step < global_step:
            continue

        encoded = {k: v.to(device) for k, v in encoded.items()}
        labels = labels.to(device)

        with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=(device.type == "cuda")):
            output = model(encoded)
            emb = output["sentence_embedding"]
            loss = cosine_loss(emb, labels)

        # Backward
        optimizer.zero_grad(set_to_none=True)
        if use_fp16:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(trainable_params, MAX_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(trainable_params, MAX_GRAD_NORM)
            optimizer.step()

        scheduler.step()
        global_step += 1
        running_loss += loss.item()

        # ── Logging ────────────────────────────────────────────────────
        if global_step % LOG_STEPS == 0:
            avg = running_loss / LOG_STEPS
            lr_now = scheduler.get_last_lr()[0]
            elapsed = time.time() - t0
            steps_sec = LOG_STEPS / elapsed if elapsed > 0 else 0
            cos_sim = 1.0 - avg
            train_losses.append((global_step, avg))
            print(
                f"  Epoch {epoch+1}/{EPOCHS}  "
                f"Step {global_step:>7,}/{total_steps:,}  "
                f"Loss: {avg:.4f}  "
                f"CosSim: {cos_sim:.4f}  "
                f"LR: {lr_now:.2e}  "
                f"Speed: {steps_sec:.1f} step/s"
            )
            running_loss = 0.0
            t0 = time.time()

        # ── Checkpoint ─────────────────────────────────────────────────
        if global_step % SAVE_STEPS == 0:
            eval_loss = evaluate()
            eval_cos = 1.0 - eval_loss
            eval_losses.append((global_step, eval_loss))
            print(f"   📊 Eval loss: {eval_loss:.4f}  |  Eval CosSim: {eval_cos:.4f}")
            save_checkpoint(epoch, global_step)
            model.train()
            t0 = time.time()  # reset timer after checkpoint overhead

    # ── End-of-epoch eval & checkpoint ─────────────────────────────────
    eval_loss = evaluate()
    eval_cos = 1.0 - eval_loss
    eval_losses.append((global_step, eval_loss))
    print(f"\n✅ Epoch {epoch+1} complete  |  Eval loss: {eval_loss:.4f}  |  CosSim: {eval_cos:.4f}")
    save_checkpoint(epoch + 1, global_step, extra_tag=f"epoch{epoch+1}")

print(f"\n{'='*70}")
print(f"  TRAINING COMPLETE  —  Final global step: {global_step}")
print(f"{'='*70}")

## 4. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ── Loss curves ────────────────────────────────────────────────────────────
if train_losses:
    steps_t, losses_t = zip(*train_losses)
    axes[0].plot(steps_t, losses_t, label="Train loss", alpha=0.7)
if eval_losses:
    steps_e, losses_e = zip(*eval_losses)
    axes[0].plot(steps_e, losses_e, "o-", label="Eval loss", markersize=4)
axes[0].set_xlabel("Global Step")
axes[0].set_ylabel("Cosine Loss")
axes[0].set_title("Loss Curves")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ── Cosine similarity curves ──────────────────────────────────────────────
if train_losses:
    axes[1].plot(steps_t, [1 - l for l in losses_t], label="Train CosSim", alpha=0.7)
if eval_losses:
    axes[1].plot(steps_e, [1 - l for l in losses_e], "o-", label="Eval CosSim", markersize=4)
axes[1].set_xlabel("Global Step")
axes[1].set_ylabel("Cosine Similarity")
axes[1].set_title("Student ↔ Teacher Cosine Similarity")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(DRIVE_ROOT / "training_curves.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"📈 Curves saved to {DRIVE_ROOT / 'training_curves.png'}")

## 5. Save Final SentenceTransformer Model

In [ ]:
# Save the final fine-tuned SentenceTransformer to Drive
model.save(str(FINAL_MODEL_DIR))

# Write export metadata
export_meta = {
    "base_model": BASE_MODEL_NAME,
    "teacher": "text-embedding-3-large",
    "output_dim": TARGET_DIM,
    "max_length": 512,
    "normalized_output": True,
    "dataset": "Qdrant/dbpedia-entities-openai3-text-embedding-3-large-1536-1M",
    "epochs": EPOCHS,
    "lr": LR,
    "final_eval_loss": eval_losses[-1][1] if eval_losses else None,
    "final_eval_cossim": 1 - eval_losses[-1][1] if eval_losses else None,
}
with open(FINAL_MODEL_DIR / "export_meta.json", "w") as f:
    json.dump(export_meta, f, indent=2)

print(f"✅ Final model saved to: {FINAL_MODEL_DIR}")
print(f"   export_meta.json: {json.dumps(export_meta, indent=2)}")

## 6. Export to ONNX + INT8 Quantisation

Export the trained model in three formats:
1. **`model.onnx`** — Full-precision float32
2. **`model.int8.onnx`** — Dynamic INT8 quantised (smaller, faster on CPU/WASM)
3. **`model.int8.ort`** — ORT-format INT8 for minimal runtime deployments

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType
from pathlib import Path

onnx_float_path = ONNX_DIR / "model.onnx"
onnx_int8_path  = ONNX_DIR / "model.int8.onnx"
onnx_ort_path   = ONNX_DIR / "model.int8.ort"

# ═══════════════════════════════════════════════════════════════════════════
# 6a. Export to ONNX (float32)
# ═══════════════════════════════════════════════════════════════════════════
model.eval()
model.to("cpu")

# Create dummy inputs
dummy_text = "This is a test sentence for ONNX export."
dummy_encoded = tokenizer(
    [dummy_text], padding="max_length", truncation=True,
    max_length=512, return_tensors="pt"
)

# Build a wrapper that takes flat tensors and returns sentence_embedding
class OnnxWrapper(nn.Module):
    def __init__(self, st_model):
        super().__init__()
        self.st_model = st_model

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        features = {"input_ids": input_ids, "attention_mask": attention_mask}
        if token_type_ids is not None:
            features["token_type_ids"] = token_type_ids
        out = self.st_model(features)
        emb = out["sentence_embedding"]
        # L2 normalise
        return F.normalize(emb, p=2, dim=-1)

wrapper = OnnxWrapper(model)
wrapper.eval()

# Determine input names and dummy inputs
input_names = ["input_ids", "attention_mask"]
dummy_args = (dummy_encoded["input_ids"], dummy_encoded["attention_mask"])
if "token_type_ids" in dummy_encoded:
    input_names.append("token_type_ids")
    dummy_args = dummy_args + (dummy_encoded["token_type_ids"],)

dynamic_axes = {name: {0: "batch", 1: "seq"} for name in input_names}
dynamic_axes["sentence_embedding"] = {0: "batch"}

torch.onnx.export(
    wrapper,
    dummy_args,
    str(onnx_float_path),
    input_names=input_names,
    output_names=["sentence_embedding"],
    dynamic_axes=dynamic_axes,
    opset_version=14,
    do_constant_folding=True,
)
print(f"✅ Float ONNX exported: {onnx_float_path}")
print(f"   Size: {onnx_float_path.stat().st_size / 1e6:.1f} MB")

# Validate
onnx_model = onnx.load(str(onnx_float_path))
onnx.checker.check_model(onnx_model)
print("   ONNX model validated ✓")

# ═══════════════════════════════════════════════════════════════════════════
# 6b. Dynamic INT8 quantisation
# ═══════════════════════════════════════════════════════════════════════════
quantize_dynamic(
    model_input=str(onnx_float_path),
    model_output=str(onnx_int8_path),
    weight_type=QuantType.QInt8,
)
print(f"\n✅ INT8 ONNX exported: {onnx_int8_path}")
print(f"   Size: {onnx_int8_path.stat().st_size / 1e6:.1f} MB")

# ═══════════════════════════════════════════════════════════════════════════
# 6c. Convert INT8 ONNX to ORT format
# ═══════════════════════════════════════════════════════════════════════════
try:
    from onnxruntime.transformers.convert_to_ort import convert_onnx_to_ort
    convert_onnx_to_ort(str(onnx_int8_path))
    # The .ort file is created alongside the .onnx file
    generated_ort = onnx_int8_path.with_suffix(".ort")
    if generated_ort.exists():
        shutil.copy2(str(generated_ort), str(onnx_ort_path))
        print(f"\n✅ INT8 ORT exported: {onnx_ort_path}")
        print(f"   Size: {onnx_ort_path.stat().st_size / 1e6:.1f} MB")
    else:
        print("⚠️  ORT conversion completed but output file not found at expected path.")
except ImportError:
    print("⚠️  ORT format conversion not available. Skipping .ort export.")
    print("   Install with: pip install onnxruntime-tools")

# ═══════════════════════════════════════════════════════════════════════════
# Copy tokenizer files to ONNX directory for self-contained deployment
# ═══════════════════════════════════════════════════════════════════════════
tokenizer_files = ["tokenizer.json", "tokenizer_config.json", "vocab.txt",
                   "special_tokens_map.json", "config.json"]
for fname in tokenizer_files:
    src = FINAL_MODEL_DIR / fname
    if not src.exists():
        # Try inside the transformer subfolder
        for sub in FINAL_MODEL_DIR.iterdir():
            if sub.is_dir():
                candidate = sub / fname
                if candidate.exists():
                    src = candidate
                    break
    if src.exists():
        shutil.copy2(str(src), str(ONNX_DIR / fname))

# Copy export_meta.json
shutil.copy2(str(FINAL_MODEL_DIR / "export_meta.json"), str(ONNX_DIR / "export_meta.json"))

print(f"\n📦 All ONNX artefacts saved to: {ONNX_DIR}")
print("   Contents:")
for f in sorted(ONNX_DIR.iterdir()):
    size = f.stat().st_size
    print(f"   {f.name:40s}  {size/1e6:.2f} MB" if size > 1e5 else f"   {f.name}")

## 7. Validate ONNX Model

Sanity-check: compare PyTorch vs ONNX outputs on a few sample texts to ensure export fidelity.

In [ ]:
import onnxruntime as ort
import numpy as np

# ── Load ONNX sessions ────────────────────────────────────────────────────
sess_float = ort.InferenceSession(str(onnx_float_path))
sess_int8  = ort.InferenceSession(str(onnx_int8_path))

test_texts = [
    "Machine learning is a branch of artificial intelligence.",
    "The Eiffel Tower is located in Paris, France.",
    "Quantum computing uses qubits instead of classical bits.",
    "Python is a popular programming language for data science.",
    "Short query",
]

# ── PyTorch reference embeddings ──────────────────────────────────────────
model.eval()
model.to("cpu")
with torch.no_grad():
    encoded = tokenizer(test_texts, padding=True, truncation=True,
                        max_length=512, return_tensors="pt")
    out = model(encoded)
    pt_embs = F.normalize(out["sentence_embedding"], p=2, dim=-1).numpy()

# ── ONNX embeddings ───────────────────────────────────────────────────────
def run_onnx(session, texts):
    enc = tokenizer(texts, padding=True, truncation=True,
                    max_length=512, return_tensors="np")
    feed = {"input_ids": enc["input_ids"].astype(np.int64),
            "attention_mask": enc["attention_mask"].astype(np.int64)}
    # Add token_type_ids if model expects it
    input_names = [i.name for i in session.get_inputs()]
    if "token_type_ids" in input_names and "token_type_ids" in enc:
        feed["token_type_ids"] = enc["token_type_ids"].astype(np.int64)
    return session.run(None, feed)[0]

onnx_float_embs = run_onnx(sess_float, test_texts)
onnx_int8_embs  = run_onnx(sess_int8, test_texts)

# ── Compare ────────────────────────────────────────────────────────────────
print(f"{'Text':<55} {'PT↔Float':>10} {'PT↔INT8':>10} {'Float↔INT8':>10}")
print("─" * 90)
for i, txt in enumerate(test_texts):
    cos_pf = np.dot(pt_embs[i], onnx_float_embs[i])
    cos_pi = np.dot(pt_embs[i], onnx_int8_embs[i])
    cos_fi = np.dot(onnx_float_embs[i], onnx_int8_embs[i])
    print(f"{txt[:53]:<55} {cos_pf:>10.6f} {cos_pi:>10.6f} {cos_fi:>10.6f}")

print(f"\n✅ Mean PT↔Float cosine: {np.mean([np.dot(pt_embs[i], onnx_float_embs[i]) for i in range(len(test_texts))]):.6f}")
print(f"✅ Mean PT↔INT8  cosine: {np.mean([np.dot(pt_embs[i], onnx_int8_embs[i]) for i in range(len(test_texts))]):.6f}")
print(f"✅ Embedding dim: {onnx_float_embs.shape[1]}")

## 8. (Optional) Push to Hugging Face Hub

Uncomment and fill in your HF token to push the model & ONNX artefacts to your Hub repo.

In [ ]:
# ── Uncomment below to push to HF Hub ─────────────────────────────────────
# from huggingface_hub import HfApi, login
#
# HF_TOKEN = "hf_YOUR_TOKEN_HERE"  # or use Colab secrets
# REPO_ID  = "your-username/mdbr-leaf-mt-1536-large-onnx"
#
# login(token=HF_TOKEN)
# api = HfApi()
#
# # Upload ONNX directory
# api.upload_folder(
#     folder_path=str(ONNX_DIR),
#     repo_id=REPO_ID,
#     repo_type="model",
#     commit_message="Upload distilled ONNX model (text-embedding-3-large, 1536d)",
# )
#
# # Upload final SentenceTransformer model
# api.upload_folder(
#     folder_path=str(FINAL_MODEL_DIR),
#     repo_id=REPO_ID,
#     path_in_repo="sentence_transformer",
#     repo_type="model",
#     commit_message="Upload SentenceTransformer checkpoint",
# )
#
# print(f"✅ Pushed to https://huggingface.co/{REPO_ID}")

print("ℹ️  Uncomment the cell above and set your HF token to push to the Hub.")